In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .set("spark.sql.shuffle.partitions", "10")
 

sc = SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

minio_ip_address = "minio"

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-afd65b36-0282-40bd-8637-4044ab8ae0bc;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType

# Kafka configuration
kafka_broker = "kafka1:9092"
kafka_topic = "planes"

# Define schema for the Kafka message
# schema = StructType([
#     StructField("created_at", StringType(), True),
#     StructField("entry_id", IntegerType(), True),
#     StructField("wind_speed", StringType(), True)  # Voltage is initially a string
# ])

# Read raw data from Kafka
raw_stream = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Parse Kafka messages
# parsed_stream = raw_stream.selectExpr("CAST(value AS STRING) AS message") \
#     .select(from_json(col("message"), schema).alias("data")) \
#     .select(
#         col("data.created_at").cast(TimestampType()).alias("created_at"),  # Convert timestamp to Spark TimestampType
#         col("data.entry_id").alias("entry_id"),
#         col("data.wind_speed").cast(FloatType()).alias("wind_speed")  # Convert voltage to FloatType for aggregation
#     ).dropDuplicates(["entry_id"]).withWatermark("created_at", "5 minutes")

# parsed_stream = parsed_stream.filter(col("data.wind_speed").isNotNull())


# # Compute rolling average over a 5-minute window
# rolling_average = parsed_stream \
#     .groupBy(window(col("created_at"), "5 minutes")) \
#     .agg(avg("wind_speed").alias("rolling_avg_wind_speed")) \
#     .select(
#         col("window.start").alias("window_start"),
#         col("window.end").alias("window_end"),
#         col("rolling_avg_wind_speed")
#     )  # Explicit ordering by window start

# # Output rolling average to the console
query = raw_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()